Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [x] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [x] Continue to participate in our Kaggle challenge. 
- [x] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [x] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [x] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](http://contrib.scikit-learn.org/category_encoders/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/category_encoders/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](http://contrib.scikit-learn.org/category_encoders/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [51]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [52]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# Random Forest
Tree-based models tend to work best with labeled and tabular data, so we'll use RandomForests.

In [53]:
# Assign x, y sets
target = 'status_group'
features = train.columns.drop(target)

X_train = train[features]
y_train = train[target]

X_test = test[features]

In [5]:
# Random forest accuracy
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

baseline_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=400, random_state=59, n_jobs=-1)
)

k = 10    # k-fold cross-validation, so k different accuracy scores
scores = cross_val_score(baseline_pipeline, X_train, y_train, cv=k, 
                         scoring='roc_auc_ovr')   # also try roc_auc_ovo
print(f'ROC AUC for {k} folds:', scores)  

ROC AUC for 10 folds: [0.90454065 0.90583549 0.90590424 0.90059825 0.90626521 0.9016745
 0.90715898 0.90105133 0.89908949 0.90857179]


In [6]:
scores.mean()

0.90406899258816

# Feature Engineering

### First Iteration
We'll iteratively tweak and test the dataset later.

In [54]:
# We'll copy train and test here, then make new copies for each iteration.
# train_1, train_2, train_3...

import numpy as np

def wrangle_1(X):
    """feature engineer train and test sets the same way"""
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # Drop recorded_by (never varies), id (always varies, random)
    # num_private has 98.6% of values == 0, no clear relationship with target
    unusable_variance = ['recorded_by', 'id'] #, 'num_private']
    X = X.drop(columns=unusable_variance)
        
    return X

# New train and test sets for first iteration
train_1 = wrangle_1(train)
test_1 = wrangle_1(test)

X_train_1 = train_1.drop(columns=target)
y_train_1 = train_1[target]
X_test_1 = test_1

In [9]:
# New pipeline for first iteration
pipeline_1 = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=400, random_state=59, n_jobs=-1) # n_estimators=100 by default
)

k = 10    # k-fold cross-validation, so k different accuracy scores
scores_1 = cross_val_score(pipeline_1, X_train_1, y_train_1, cv=k, 
                         scoring='roc_auc_ovr')
print(f'ROC AUC for {k} folds:', scores_1)

ROC AUC for 10 folds: [0.90316262 0.90493012 0.90459689 0.89858081 0.90479803 0.89873708
 0.90523655 0.90062876 0.8965959  0.90915481]


In [10]:
scores_1.mean()

0.9026421580153198

We see a slight decrease in accuracy, but `recorded_by` cannot be useful (since it's a constant), and `id` is almost certainly not useful (since it's probably random), so we'll proceed.

In [11]:
# Randomized search
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
final_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=250, random_state=61, n_jobs=-1, max_features=None) # n_estimators=100 by default
)


param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__criterion': ['gini', 'entropy'] 
}

# Instantiate this class
# If you're on Colab, decrease n_iter & cv parameters
search_final = RandomizedSearchCV(
    final_pipeline, 
    param_distributions=param_distributions, 
    n_iter=10,   # n_iter=10 by default
    cv=5,        # cv=5 by default
    scoring='roc_auc_ovr', 
    verbose=10, # verbose diagnostics. step-by-step progress reports
    return_train_score=True, 
    n_jobs=-1
)

search_final.fit(X_train_1, y_train_1);

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Users/RyanKoul/opt/anaconda3/envs/unit2/lib/python3.7/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed: 29.2min remaining:  7.3min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 37.2min finished


In [12]:
# Attributes are available after randomized search (cross validation) is done
print('Best hyperparameters', search_final.best_params_)
print('Cross-validation ROC AUC', search_final.best_score_)

Best hyperparameters {'simpleimputer__strategy': 'mean', 'randomforestclassifier__criterion': 'entropy'}
Cross-validation ROC AUC 0.8954431453272516


In [15]:
# search_final
# y_pred2 = search_final.predict(X_test)
# final_pipeline.fit(X_train, y_train)
y_pred_final = search_final.predict(X_test_1)

submission4 = sample_submission.copy()
submission4['status_group'] = y_pred_final
submission4.to_csv('koul4.csv', index=False)

## Other Iterations
This feature engineering did not improve the model's predictive power, but I leave it here for reference.

In [55]:
# import random

# def randomize_null(s):
#     mask = s.isna()
#     return pd.Series(np.where(mask, np.random.choice(s[~mask], len(s)), s))

def wrangle_2(X):
    
    X = X[:]
    # X['population'] = randomize_null(X['population'])
    
    X = X.drop(columns='num_private')
    
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
      # X[col] = randomize_null(X[col])

    return X

train_2 = wrangle_2(train_1)
test_2 = wrangle_2(test_1)

X_train_2 = train_2.drop(columns=target)
y_train_2 = train_2[target]
X_test_2 = test_2

In [42]:
pipeline_2 = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=100, random_state=59, n_jobs=-1) # n_estimators=100 by default
)

k = 5    # k-fold cross-validation, so k different accuracy scores
scores_2 = cross_val_score(pipeline_2, X_train_2, y_train_2, cv=k, 
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores_2)

Accuracy for 5 folds: [0.81119529 0.80664983 0.80968013 0.81060606 0.80799663]


In [43]:
scores_2.mean()

0.8092255892255892

In [56]:
def wrangle_3(X):
    
    X = X[:]
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    duplicates = ['quantity_group', 'payment_type', 'waterpoint_type_group']
    X = X.drop(columns=duplicates)
    
    # More likely to be non functional
    X['funded_by_Tanzania_gov'] = X['funder'] == 'Government Of Tanzania'
    X['not_funded_by_Tanzania_gov'] = X['funder'] != 'Government of Tanzania'
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years_since'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years_since'].isnull()
    
    X['wpt_name'] = X['wpt_name'].replace('none', None)
    X['wpt_name_MISSING'] = X['wpt_name'].isnull()
    
    return X

train_3 = wrangle_3(train_2)
test_3 = wrangle_3(test_2)

X_train_3 = train_3.drop(columns=target)
y_train_3 = train_3[target]
X_test_3 = test_3

In [57]:
pipeline_3 = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=100, random_state=59, n_jobs=-1) # n_estimators=100 by default
)

k = 5    # k-fold cross-validation, so k different accuracy scores
scores_3 = cross_val_score(pipeline_3, X_train_3, y_train_3, cv=k, 
                         scoring='accuracy')
print(f'Accuracy for {k} folds:', scores_3)

Accuracy for 5 folds: [0.81026936 0.80917508 0.80690236 0.80858586 0.80513468]


In [58]:
scores_3.mean()

0.8080134680134681

### Other, unfinished feature engineering

In [ ]:
def wrangle_4(X):
    
    X = X[:]
    
    big_funders = ['Danida', 'HESAWA', 'Rwssp','World Bank', 'Kkkt', 'World Vision', 'Unicef', 'Tasaf']
    X['big_funders'] = X['funder'].isin(big_funders)
    X['other_funders'] = ((~train['funder'].isin(big_funders)) & (train['funder'] != 'Government Of Tanzania'))
    

    # At locations where the funder is NOT in the top 10, 
    # replace it with 'OTHER'
    top10funders = X['funder'].value_counts()[:10].index
    X.loc[~X['funder'].isin(top10funders), 'funder'] = 'OTHER'
    
    top20installers = X['installer'].value_counts()[:10].index
    X.loc[~X['installer'].isin(top10funders), 'installer'] = 'OTHER'
    
    
    # Very high cardinality
    X = X.drop(columns='subvillage')
    
    return X

In the data dictionary, `amount_tsh` is the total static head (amount water available to waterpoint). This seems like missing data; since there are functional water pumps when `amount_tsh` == 0, there must be some water available. 

Another definition of static head is "total vertical distance that a pump raises water".

`amount_tsh` is 70% zeros, so replacing them with np.NaNs and imputing the median wouldn't change anything.
Let's see how often these 0 values show up in our target classes. This will help us see the relationship between the variables.

In [ ]:
import numpy as np

# group data by amount_tsh 0 vs not 0 (binary variable). find the % of missing values
# in amount_tsh per class in status_group. Are they more common in some classes?

train['amount_tsh_null'] = train['amount_tsh'].replace(0, np.NaN).isnull()

print(train['amount_tsh_null'].groupby(train['status_group']).sum())
train['amount_tsh_null'].groupby(train['status_group']).mean()

We see that `amount_tsh` is much more likely to be missing in `non_functional` status group than in the `functional` one but is nearly equal in counts. We'll keep our newly created column `amount_tsh_null` and drop the original.

In [ ]:
train['amount_tsh_not_null'] = train['amount_tsh'].replace(0, np.NaN).notna()

print(train['amount_tsh_not_null'].groupby(train['status_group']).sum())
train['amount_tsh_not_null'].groupby(train['status_group']).mean()

While counts are more different, we see the opposite pattern for not null, since this is just the complement of the previous probabilities. Rather than encode non-nulls separately, we can group them together since there doens't appear to much of a difference.

### Relationships between NaNs in columns and target labels

In [ ]:
# % of pumps funded by World Bank in each status group

# More likely to be non functional
(train['funder'] == 'World Bank').groupby(train['status_group']).mean()

In [ ]:
top10 = train['funder'].value_counts()[:10].index
big_funders = ['Danida', 'HESAWA', 'Rwssp',
'World Bank', 'Kkkt', 'World Vision', 'Unicef', 'Tasaf']
train['funder'].isin(big_funders).groupby(train['status_group']).mean()

(train.loc[~train['funder'].isin(top10), 'funder'] = 'OTHER')

In [ ]:
#  % of missing values in gps_height for each type of pump.

# Slightly related to functional needs repair
train['gps_height'].replace(0, np.NaN).isnull().groupby(train['status_group']).mean()
train['gps_height'].value_countsounts()

In [ ]:
# % of missing values in installer for each type of pump.
train['installer'].isnull().groupby(train['status_group']).mean()

In [ ]:
# Less functional
big_installers = ['Government', 'RWE', 'Commu', 'DANIDA']
train['installer'].isin(big_installers).groupby(train['status_group']).mean()

# biggest installer
(train['installer'] == 'DWE').groupby(train['status_group']).mean()

In [ ]:
# Higher % of unnamed pumps in functional than in others
# 'wpt_name_MISSING'
train['wpt_name'].replace('none', np.NaN).isnull().groupby(train['status_group']).mean()

### Last iteration, for now

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# dict whose keys are parameters to vary, values are their values
# Menu of options to randomly choose from
# alpha parameter (penalty on coefficients). A little or a lot of regularization?

final_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=95, n_jobs=-1, max_features=None) # n_estimators=100 by default
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__criterion': ['gini', 'entropy'], 
}

# Instantiate this class
# If you're on Colab, decrease n_iter & cv parameters
search_final = RandomizedSearchCV(
    final_pipeline, 
    param_distributions=param_distributions, 
    n_iter=100,   # n_iter=10 by default
    cv=10,        # cv=5 by default
    scoring='roc_auc_ovr', 
    verbose=10, # verbose diagnostics. step-by-step progress reports
    return_train_score=True, 
    n_jobs=-1
)

search_final.fit(X_train_1, y_train_1);

In [ ]:
# Attributes are available after randomized search (cross validation) is done
print('Best hyperparameters', search_final.best_params_)
print('Cross-validation ROC AUC', search_final.best_score_)